In [1]:
import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
#conda install -c anaconda beautifulsoup4


In [2]:
from bs4 import BeautifulSoup
Canada_data = BeautifulSoup(website_url, 'html.parser')

In [3]:
column_names=['PostalCode','Borough','Neighborhood']
city=pd.DataFrame(columns = column_names)

# Parse the file to get columns

In [4]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    city = city.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

Cleaned the data based on Postal code and Borough.

1.Drop "None" rows in DataFrame.
2.Drop any row which contains 'Not assigned' value.
3.All "Not assigned" will be replace to 'NaN' using numpy for convenience.

In [5]:
#clean
city = city[city.Borough!='Not assigned']
city = city[city.Borough!= 0]
city.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,city.shape[0]):
    if city.iloc[i][2] == 'Not assigned':
        city.iloc[i][2] = city.iloc[i][1]
        i = i+1
 
#city['Neighborhood'].value_counts()
df = city.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Now we will drop null values from the dataframe

In [6]:

df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df.shape

(103, 3)

In [8]:
df.to_csv('data.csv')